# Overview

In this notebook we are going to load data from our machine into a spark cluster.

## Prerequisites
It assumes you already have a running spark cluster. In our case we have prepared our spark cluster to run on kubernetes. If you haven't done so already, read through the following notebooks to get setup:
- [Install Apache Spark Prerequisites](Install%20Apache%20Spark%20Prerequisites.ipynb)
- [Running Apache Spark On Kubernetes](Running%20Apache%20Spark%20On%20Kubernetes.ipynb)
- [Intro To Koalas](Intro%20To%20Koalas.ipynb)

Note: We will see that the instructions are basically the same as [Load CSV Into Apache Spark Locally](Load%20CSV%20Into%20Apache%20Spark%20Locally.ipynb) once you get the kubernetes stuff setup.

## Adjenda
1. Understand Architecture
2. Create SparkContext
3. Setup Datastore
4. Load Data
5. Cleanup Spark and Kubernetes

# 1. Understand Atchitecture

Before we start working, we need to understand a few things related to architecture.

If we think of the spark cluster as a server and our jupyter noteboook as a client we will see that there are two places from which data can be loaded. 
- Option 1: We load data from a file on the client
- Option 2: We load data from a file on the workers

Currently the Spark/Koalas framework provides a means for accomplishing both options but each have their own caveats which I will briefly describe. 

With Option 1, the process of loading the data is to create a pandas dataframe, and then create a koalas dataframe from it. The code would look something like this:
```python
import pandas
from databricks import koalas

pdf = pandas.read_csv(...)
kdf = koalas.DataFrame(pdf)
```

But, in order to create the pandas dataframe, we will need to be able to load all the data into memory which is a deal breaker when working with big data; we simply do not have enough memory to load such a large dataset on one machine.

With Option 2, the process of loading data is to instruct the spark workers to load a file from their local filesystem. The Caveat is that Spark has made an assumption that the driver and all of the workers have access to the same network file system mounted in the same place. I think this stems back to the HDFS days but I am not 100% sure. In our case, which is often the case, we do not have such a thing setup. Instead we will have to hack some utility function together. First we will need to move data from our *Example Data Sets* directory to the file system root (in our case symlink). Then take advantage of a Spark utility which will run a job which downloads a file from a URL to the root of the local filesystem on all the spark workers. Finally We will setup a simple http server to serve local files to the spark cluster. 

**Note**: It should be noted that this is strictly for educational purposes and is not inteded for production use. For production use, configure the Spark Worers to mount the network storage solution. This avoids data duplication and cuts down on the wall time because the data has less hops.

# 2. Create SparkContext
The spark context is the object which allows us interact with the spark cluster and submit jobs etc.

In [1]:
import pyprojroot
project_root_dir  = pyprojroot.here()
print(project_root_dir)

/root/ml-training-jupyter-notebooks


In [2]:
# Load a helper module
import os
import importlib.util
module_name = "spark_helper"
module_dir = os.path.join(project_root_dir, "Utilities", "{0}.py".format(module_name))
if not os.path.exists(module_dir):
    print("The helper module does not exist")
print("Loading module: {0}".format(module_dir))
spec = importlib.util.spec_from_file_location(module_name, module_dir)
spark_helper = importlib.util.module_from_spec(spec)
spec.loader.exec_module(spark_helper)

Loading module: /root/ml-training-jupyter-notebooks/Utilities/spark_helper.py


In [3]:
spark_app_name = "spark-jupyter-mlib"
docker_image = "tschneider/pyspark:v5"
k8_master_ip = "15.4.7.11"
spark_session = spark_helper.create_spark_session(spark_app_name, docker_image, k8_master_ip)
sc = spark_session.sparkContext

Setting SPARK_HOME
/opt/spark

Running findspark.init() function
['/opt/spark/python', '/opt/spark/python/lib/py4j-0.10.9-src.zip', '/usr/lib64/python36.zip', '/usr/lib64/python3.6', '/usr/lib64/python3.6/lib-dynload', '', '/usr/local/lib64/python3.6/site-packages', '/usr/local/lib/python3.6/site-packages', '/usr/lib64/python3.6/site-packages', '/usr/lib/python3.6/site-packages', '/usr/local/lib/python3.6/site-packages/IPython/extensions', '/root/.ipython']

Setting PYSPARK_PYTHON
/usr/bin/python3

Configuring URL for kubernetes master
k8s://https://15.4.7.11:6443

Determining IP Of Server
The ip was detected as: 15.4.12.12

Creating SparkConf Object
('spark.master', 'k8s://https://15.4.7.11:6443')
('spark.app.name', 'spark-jupyter-mlib')
('spark.submit.deploy.mode', 'cluster')
('spark.kubernetes.container.image', 'tschneider/pyspark:v5')
('spark.kubernetes.namespace', 'spark')
('spark.kubernetes.pyspark.pythonVersion', '3')
('spark.kubernetes.authenticate.driver.serviceAccountName', '

**Note:** We can look at kubernetes to see that out worker nodes were created.

In [4]:
! kubectl -n spark get pod

NAME                                         READY     STATUS    RESTARTS   AGE
spark-jupyter-mlib-f768867dc4b6db7c-exec-1   1/1       Running   0          18s
spark-jupyter-mlib-f768867dc4b6db7c-exec-2   1/1       Running   0          18s
spark-jupyter-mlib-f768867dc4b6db7c-exec-3   1/1       Running   0          18s


# 3. Setup Datastore
As mentioned earlier, we are going to create a webserver to serve files from our local machine to the spark cluster. In order to be able to run the web server, without having the jupyter cell run forever and prevent us from moving on with out work, we will run it in a separate thread. If you are not familiar with threads etc I suggest reading up. The key things to know are that the web server will run as a separate process and python doesn't provide an out of the box way to kill the thread. If you fudged the configs... restart the kernel to kill the web server.

This web server will be configured to serve all files from a given *web_root* directory. In this demo, I set the directory to the *./Example Data Sets* folder at the root of this project.

**Note**: This is just for testing and small scale EDA. This is not intended for production use cases!

## 3.1. Symlink Local Data Files

In [5]:
data_dir_name = "Example Data Sets"
data_dir_path = os.path.join(project_root_dir, data_dir_name)
spark_helper.link_data_dir_to_root(data_dir_path)

Creating Symlink: /root/ml-training-jupyter-notebooks/Example Data Sets/Test Scores.csv -> /Test Scores.csv
Creating Symlink: /root/ml-training-jupyter-notebooks/Example Data Sets/nasdaq_2019.csv -> /nasdaq_2019.csv
Creating Symlink: /root/ml-training-jupyter-notebooks/Example Data Sets/results.csv -> /results.csv


## 3.2. Load Web Server Module

In [6]:
# Import the module for the web server we wrote
import importlib.util
spec = importlib.util.spec_from_file_location("PythonHttpFileServer", "../../../Utilities/PythonHttpFileServer.py")
PythonHttpFileServer = importlib.util.module_from_spec(spec)
spec.loader.exec_module(PythonHttpFileServer)

## 3.3. Start Web Server
**Note**: When setting a web root, or working with files, keep in mind that URLs need to escape special characters. I have set my webroot to the Example Data Directory rather than the project root so I dont have to escape anything. That is also why I have named the file with the characters I that I did.

In [ ]:
import os

data_dir_name = "Example Data Sets"
web_root = os.path.join(project_root_dir, data_dir_name)

if not os.path.exists(web_root):
    raise Exception("The web root for the server does not exist.")

csv_file_name = "nasdaq_2019.csv"
csv_file_path = os.path.join(web_root, csv_file_name)

if not os.path.exists(csv_file_path):
    raise Exception("The data file does not exist.")
    
print("Web root and data file exist!")
print("web root: {0}".format(web_root))
print("data file: {0}".format(csv_file_path))

In [ ]:
# Import the library
import threading

# Configure the logger and log level (incase we need/want to debug)
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create and start the thread if it doesnt exist
var_exists = 'web_server_thread' in locals() or 'web_server_thread' in globals()
if not var_exists:
    web_server_port = 80
    web_server_args = (web_server_port, web_root)
    web_server_thread = threading.Thread(target=PythonHttpFileServer.run_server, args=web_server_args)
    web_server_thread.start()
else:
    print("Web Server thread already exists")
    print("To kill it you need to restart the kernel.")

# 4. Load The Data

Loading the data is not as intuitive as one would think. We Instruct the spark cluster to download a file from the web server. But when we do this, the file is not actually downloaded. Remember, spark is lazy. Instead, a link to the url is stored in the spark session object so that the file can be downloaded when we need to perform an operation on it.

Later, we will tell spark to return a handle to a dataframe consisting of the data contained in this url. At that point, lazy spark, will download the data file to each worker (at their root), open the file, and distribute the data accross the cluster. We will see proof of this when we call the addFile() function. Our server logs will show that the workers are making web requests to it.

## 4.1. Add the file using Spark Context
We will use the *addFiles()* function available on the SparkContect object to download a file to the workers. Behind the scenes, this file submits a job to the cluster. So the file isnt actually downloaded until I do some work. This is made apparent when the server logs pop up in a jupyter cell below.

In [ ]:
ip_address = spark_helper.determine_ip_address()
csv_file_url = "http://{0}:{1}/{2}".format(ip_address, web_server_port, csv_file_name)
print("Uploading file '{0}' to Spark cluster.".format(csv_file_url))
sc.addFile(csv_file_url)

## 4.2. Use koalas to open the file on spark

Import the utility function to convert a date string to a datetime object from our utilities module

In [ ]:
# Import the utilities module we wrote
import importlib.util
spec = importlib.util.spec_from_file_location("utilities", "../../../Utilities/utilities.py")
utilities = importlib.util.module_from_spec(spec)
spec.loader.exec_module(utilities)

# Define a mapping to convert our data field to the correct type
converter_mapping = {
    "date": utilities.convert_date_string_to_date
}

Load our OHCLV data Into a koalas dataframe and pull out a single day in the say way we would in pandas

In [ ]:
# Avoid a warning
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

from databricks import koalas
koalas_dataframe = koalas.read_csv(u"file:///nasdaq_2019.csv", converters=converter_mapping)

In [ ]:
koalas_dataframe.head()

We should see the workers download the file in the logs. If we log into the nodes we can see the file is located on the filesystem root.

In [ ]:
! kubectl -n spark get pods

In [ ]:
! kubectl -n spark exec -ti spark-jupyter-mlib-51eab07dc4a02d17-exec-1 -- find / -name nasdaq_2019.csv

# 5. Cleanup

In [ ]:
sc.stop()

In [ ]:
! kubectl -n spark get pod

In [ ]:
spark_helper.unlink_data_dir_from_root(data_dir_path)